In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("2019_legis_data_4.12.2021.csv",sep="|")

In [3]:
df_dtg = df[df.dtg == True]

In [4]:
df.columns

Index(['bill_number', 'session', 'url', 'bill_title', 'spon_sen_name',
       'dist_name', 'aye_vote', 'nay_vote', 'dtg', 'dtg_date', 'lla_needed?',
       'lla_date', 'lla_action', 'signed_by_gov', 'signed_date',
       'vetoed_by_gov', 'vetoed_date', 'sponser_memo', 'sponser_memo2',
       'bill_text'],
      dtype='object')

In [5]:
df_signed = df[df.signed_by_gov == True]
df_vetoed = df[df.vetoed_by_gov == True]

### Vectorize

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
vectorizer = TfidfVectorizer(stop_words='english')
df = df.dropna()
matrix = vectorizer.fit_transform(df.bill_text)
bill_text_vectorized_df = pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names())

bill_text_vectorized_df

,00,000,0000,0001,00010,00011,00016207,0002,0003,00033784,...,zoster,zpass,zte,zucherro,zweig,zygote,zz,zzz,zzzz,zzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


But what I want to do is find a correlation between length of time to sign and content of the bill. Perhaps I need to ask Adrian about some kind of regression here.

## Random Forest Classifers for signed bills

In [8]:
# df['is_signed'] = (df['signed_by_gov'] == True).astype(int)
# df['is_vetoed'] = (df['vetoed_by_gov'] == True).astype(int)

#### Only signed stuff for this section

In [9]:
# X = bill_text_vectorized_df
# y = df.is_signed

In [10]:
# from sklearn.ensemble import RandomForestClassifier

# clf = RandomForestClassifier(n_estimators=100)
# clf.fit(X,y)

In [11]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X,y)

In [12]:
# from sklearn.metrics import confusion_matrix

# y_true = y_test
# y_pred = clf.predict(X_test)
# matrix = confusion_matrix(y_true, y_pred)

# label_names = pd.Series(['signed','not signed'])
# pd.DataFrame(matrix,
#             columns='Predicted ' + label_names,
#             index='Is ' + label_names)

In [13]:
# clf.score(X,y)

In [14]:
# clf.score(X_test,y_test)

## Regression Setup

What I'd like to do here is set up a linear regression that attempts to find a relationship between the content of a bill and the length of time it took to get delivered to the governor's office.

I suspect there's a link between the content and the speed.

In [15]:
df_dtg.columns

Index(['bill_number', 'session', 'url', 'bill_title', 'spon_sen_name',
       'dist_name', 'aye_vote', 'nay_vote', 'dtg', 'dtg_date', 'lla_needed?',
       'lla_date', 'lla_action', 'signed_by_gov', 'signed_date',
       'vetoed_by_gov', 'vetoed_date', 'sponser_memo', 'sponser_memo2',
       'bill_text'],
      dtype='object')

In [16]:
df_dtg.signed_by_gov.values

array([False, False, False, ...,  True,  True,  True])

Remove the one error string

In [17]:
df_dtg = df_dtg[~df_dtg.lla_date.str.contains('<class \'Exception\'>')]

Convert to datetime

In [18]:
df_dtg['lla_date'] = pd.to_datetime(df_dtg['lla_date']).dt.day
df_dtg['dtg_date'] = pd.to_datetime(df_dtg['dtg_date']).dt.day

In [19]:
df_dtg.head()

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,lla_needed?,lla_date,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text
12,senate Bill S23,2019,https://www.nysenate.gov/legislation/bills/201...,Relates to maintaining the continued viability...,Kevin S. Parker,"(D, WF) 21st Senate District",62,0,True,6,True,20,returned to senate,False,no date,True,"Dec 13, 2019",BILL NUMBER: S23\n\nSPONSOR: PARKER\n \nTITLE ...,"program. Legacy large-scale renewables (LSRs),...",S23 (ACTIVE) - Bill Text\n \ndownload...
14,senate Bill S25B,2019,https://www.nysenate.gov/legislation/bills/201...,"Enacts the ""bird-friendly building council act...",Brad Hoylman,"(D, WF) 27th Senate District",43,18,True,8,True,18,returned to senate,False,no date,True,"Nov 20, 2019",BILL NUMBER: S25\n\nSPONSOR: HOYLMAN\n \nTITLE...,existing buildings that pose a threat for coll...,S25 - Bill Text\n \ndownload pdf\n\n\...
20,senate Bill S31,2019,https://www.nysenate.gov/legislation/bills/201...,Requires organizations that receive funds for ...,Brad Hoylman,"(D, WF) 27th Senate District",62,0,True,26,True,20,returned to assembly,False,no date,True,"Dec 06, 2019",BILL NUMBER: S31\n\nSPONSOR: HOYLMAN\n \nTITLE...,JUSTIFICATION:\n\nAlthough Energy Star product...,S31 (ACTIVE) - Bill Text\n \ndownload...
30,senate Bill S41,2019,https://www.nysenate.gov/legislation/bills/201...,Permits antitrust fines or penalties to be pai...,Brad Hoylman,"(D, WF) 27th Senate District",55,5,True,13,True,9,returned to senate,True,"Sep 13, 2019",False,no date,BILL NUMBER: S41\n\nSPONSOR: HOYLMAN\n \nTITLE...,JUSTIFICATION:\n\nAntitrust crime by its very ...,S41 (ACTIVE) - Bill Text\n \ndownload...
34,senate Bill S45B,2019,https://www.nysenate.gov/legislation/bills/201...,Relates to discharged LGBT veterans,Brad Hoylman,"(D, WF) 27th Senate District",61,0,True,8,True,18,returned to senate,True,"Nov 12, 2019",False,no date,BILL NUMBER: S45\n\nSPONSOR: HOYLMAN\n \nTITLE...,-New section 369-1 of the executive law establ...,S45 - Bill Text\n \ndownload pdf\n\n\...


In [20]:
df_dtg.dtypes

bill_number      object
session           int64
url              object
bill_title       object
spon_sen_name    object
dist_name        object
aye_vote         object
nay_vote         object
dtg                bool
dtg_date          int64
lla_needed?        bool
lla_date          int64
lla_action       object
signed_by_gov      bool
signed_date      object
vetoed_by_gov      bool
vetoed_date      object
sponser_memo     object
sponser_memo2    object
bill_text        object
dtype: object

Create the 'time_to_dtg' column

In [21]:
df_dtg['time_to_dtg'] = df_dtg['dtg_date'] - df_dtg['lla_date']

In [22]:
df_dtg['signed_by_gov_as_int'] = df_dtg.signed_by_gov.astype(int)

In [23]:
df_dtg.head()

,bill_number,session,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,dtg,dtg_date,...,lla_action,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,time_to_dtg,signed_by_gov_as_int
12,senate Bill S23,2019,https://www.nysenate.gov/legislation/bills/201...,Relates to maintaining the continued viability...,Kevin S. Parker,"(D, WF) 21st Senate District",62,0,True,6,...,returned to senate,False,no date,True,"Dec 13, 2019",BILL NUMBER: S23\n\nSPONSOR: PARKER\n \nTITLE ...,"program. Legacy large-scale renewables (LSRs),...",S23 (ACTIVE) - Bill Text\n \ndownload...,-14,0
14,senate Bill S25B,2019,https://www.nysenate.gov/legislation/bills/201...,"Enacts the ""bird-friendly building council act...",Brad Hoylman,"(D, WF) 27th Senate District",43,18,True,8,...,returned to senate,False,no date,True,"Nov 20, 2019",BILL NUMBER: S25\n\nSPONSOR: HOYLMAN\n \nTITLE...,existing buildings that pose a threat for coll...,S25 - Bill Text\n \ndownload pdf\n\n\...,-10,0
20,senate Bill S31,2019,https://www.nysenate.gov/legislation/bills/201...,Requires organizations that receive funds for ...,Brad Hoylman,"(D, WF) 27th Senate District",62,0,True,26,...,returned to assembly,False,no date,True,"Dec 06, 2019",BILL NUMBER: S31\n\nSPONSOR: HOYLMAN\n \nTITLE...,JUSTIFICATION:\n\nAlthough Energy Star product...,S31 (ACTIVE) - Bill Text\n \ndownload...,6,0
30,senate Bill S41,2019,https://www.nysenate.gov/legislation/bills/201...,Permits antitrust fines or penalties to be pai...,Brad Hoylman,"(D, WF) 27th Senate District",55,5,True,13,...,returned to senate,True,"Sep 13, 2019",False,no date,BILL NUMBER: S41\n\nSPONSOR: HOYLMAN\n \nTITLE...,JUSTIFICATION:\n\nAntitrust crime by its very ...,S41 (ACTIVE) - Bill Text\n \ndownload...,4,1
34,senate Bill S45B,2019,https://www.nysenate.gov/legislation/bills/201...,Relates to discharged LGBT veterans,Brad Hoylman,"(D, WF) 27th Senate District",61,0,True,8,...,returned to senate,True,"Nov 12, 2019",False,no date,BILL NUMBER: S45\n\nSPONSOR: HOYLMAN\n \nTITLE...,-New section 369-1 of the executive law establ...,S45 - Bill Text\n \ndownload pdf\n\n\...,-10,1


In [24]:
df_dtg.dtypes

bill_number             object
session                  int64
url                     object
bill_title              object
spon_sen_name           object
dist_name               object
aye_vote                object
nay_vote                object
dtg                       bool
dtg_date                 int64
lla_needed?               bool
lla_date                 int64
lla_action              object
signed_by_gov             bool
signed_date             object
vetoed_by_gov             bool
vetoed_date             object
sponser_memo            object
sponser_memo2           object
bill_text               object
time_to_dtg              int64
signed_by_gov_as_int     int32
dtype: object

In [25]:
vectorizer = TfidfVectorizer(stop_words='english')
df_dtg = df_dtg.dropna()
matrix = vectorizer.fit_transform(df_dtg.bill_text)
bill_text_vectorized_df = pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names())

bill_text_vectorized_df

,00,000,0000,0001,00010,00016207,0002,0003,00033784,00033893,...,zoned,zones,zoning,zoster,zucherro,zweig,zygote,zz,zzz,zzzz
0,0.0,0.005239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1329,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1330,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1331,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## TO DO

So...I'm trying to figure out a way to run a linear regression to determine how long it takes a bill to get to the governor's office based on it's content.

Right now, I'm trying to take a vectorized representation of bill text compared against the length of time it takes to get delivered to the governor.

In [27]:
import statsmodels.formula.api as smf

model = smf.logit('bill_text_vectorized_df ~ time_to_dtg',data=df_dtg)
results = model.fit()
results.summary()

PatsyError: Index mismatch between data.index and bill_text_vectorized_df
    bill_text_vectorized_df ~ time_to_dtg
    ^^^^^^^^^^^^^^^^^^^^^^^